# CPA Device Flow (User Mode)

In this step we show how to:

* Register a new client with the authorization provider.
* Associate the client with a user account.
* Obtain an access token for use with a service provider.
* Make a request to the service provider using the access token.

## Setup

Execute the following line to load the libraries


In [ ]:
require('/opt/notebooks/tutorial/lib/tutorial-utils')(this);

## Making an unauthenticated request

Initially, the client does not have an access token, so will receive a `401 Unauthorized` error when making a request to the service provider. The following request uses the RadioTAG protocol.

![Unauthenticated request](/notebooks/tutorial/images/uml/unauthenticated-request.png)

In [ ]:
this.post("http://sp:8002/radiodns/tag/1/tag", {
  bearer: "0.c221.ce15.ce1.dab",
  time: "20150909T10:00:00Z",
  time_source: "user"
}, { form: true });

## WWW-Authenticate header

The `401 Unauthorized` response contains a `WWW-Authenticate` header indicating that CPA authentication is supported by this service provider. The header contains several values:

* `version`: The protocol version in use, currently 1.0.
* `name`: The name of the authorization provider.
* `uri`: The base URL of the authorization provider.
* `modes`: A comma-separated list of available authorization modes.

## User mode

![User mode](/notebooks/tutorial/images/uml/user-mode.png)

## Registering the client

To register with the authorization provider, the client makes a `POST /request` request with the following parameters:

* `client_name`: human-readable name of the client.
* `software_id`: an identifier for the client software
    application.
* `software_version`: a version identifier for the client software application.

The URL is constructed by appending `/register` to the `uri` value from the `WWW-Authenticate` header.

In [ ]:
this.post("http://ap:8001/register", {
  client_name: "My test client",
  software_id: "cpa-test-client",
  software_version: "1.0"
});

## Associating the client with a user account

To associate with a user account, the client makes a `POST /associate` request with the following parameters:

* `client_id`: The unique identifier for the client given by the authorization provider.
* `client_secret`: A shared secret value between the client and authorization provider.
* `domain`: The service provider domain.

In [ ]:
this.post("http://ap:8001/associate", {
  client_id: "<client_id>",
  client_secret: "<client_secret>",
  domain: "sp:8002"
});

## Obtaining an access token

To obtain an access token, the client makes `POST /token` requests with the following parameters:

* `grant_type`: The value `"http://tech.ebu.ch/cpa/1.0/device_code"`.
* `client_id`: The unique identifier for the client given by the authorization provider.
* `client_secret`: A shared secret value between the client and authorization provider.
* `device_code`: The temporary device verification code.
* `domain`: The service provider domain.

In [ ]:
this.post("http://ap:8001/token", {
  grant_type: "http://tech.ebu.ch/cpa/1.0/device_code",
  client_id: "<client_id>",
  client_secret: "<client_secret>",
  device_code: "<device_code>",
  domain: "sp:8002"
});

## Approving access

To approve access, visit the verification URL, sign in, and enter the user code. These verification URL and user code values are returned by the authorization provider in the `POST /associate` response.


## Using the access token

Now that we have obtained an access token for this service provider, we can repeat our initial RadioTAG request, including the token in a `WWW-Authenticate` header.

![Authenticated request](/notebooks/tutorial/images/uml/authenticated-request.png)

In [ ]:
this.post("http://sp:8002/radiodns/tag/1/tag", {
  bearer: "0.c221.ce15.ce1.dab",
  time: "20150909T10:00:00Z",
  time_source: "user"
}, { form: true, token: "<access_token>" });

## Next

Go to [Device Flow (Client Mode)](03_CPA_Tutorial_Device_Flow_Client_Mode.ipynb).